In [20]:
import os

import tensorflow as tf
import numpy as np
import glob
import argparse
from classes.PGGAN import PGGAN
from utils.callbacks import WandbImagesPGGAN
import wandb
import tensorflow.keras as keras
from os.path import join as opj
from wandb.keras import WandbCallback
from imutils import paths
from tensorflow.data import AUTOTUNE



In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:

wandb.login()

checkpoint_path= "models/PGGAN_celebA"
config={"dataset":"celebA", "type":"PG-GAN"}
config["nota"]="DGX"
wandb.init(project="TorVergataExperiment-Generative",config=config)

images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.8/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [4]:
BS_list = [512,512,256,128,64,32]

BS=BS_list[0]

In [5]:

NOISE_DIM = 256
# Set the number of batches, epochs and steps for trainining.
# Look 800k images(16x50x1000) per each lavel
EPOCHS_PER_RES = 50


## Dataloaders

In [6]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image

In [35]:
## INIT

def read(img):
    image = tf.io.read_file(img)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def resize_and_normalize(img,target_size):
    return tf.image.resize(img,target_size)/255.

def resize(img,target_size=(4,4)):
    image = tf.io.read_file(img)
    image = tf.image.decode_jpeg(image, channels=3)
    return tf.image.resize(image,target_size)/255.

In [21]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 138545
[VALIDATION]	 17318
[TEST]		 17319


In [22]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .shuffle(1024)
    .map(resize)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .shuffle(1024)
    .map(resize)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .shuffle(1024)
    .map(resize)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [23]:
# Instantiate the optimizer for both networks
# learning_rate will be equalized per each layers by the WeightScaling scheme
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)

pgan = PGGAN(
    latent_dim = NOISE_DIM,
    d_steps = 1,
)

callbacks=[WandbImagesPGGAN(),WandbCallback()]

pgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
)

os.makedirs(checkpoint_path,exist_ok=True)


Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 pixel_normalization (PixelN  (None, 256)              0         
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 8192)              2097152   
                                                                 
 weight_scaling_4 (WeightSca  (None, 8192)             0         
 ling)                                                           
                                                                 
 bias_4 (Bias)               (None, 8192)              8192      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 8192)              0 

In [24]:
os.makedirs(checkpoint_path,exist_ok=True)
# Start training the initial generator and discriminator
pgan.fit(train_dataset, steps_per_epoch = ts, epochs = EPOCHS_PER_RES, callbacks=callbacks)
pgan.save_weights(opj(checkpoint_path, f"checkpoint_path_ndepth_0_weights_celebA.h5"))

tf.keras.utils.plot_model(pgan.generator, to_file=opj(checkpoint_path,f'generator_{pgan.n_depth}.png'), show_shapes=True)
tf.keras.utils.plot_model(pgan.discriminator, to_file=opj(checkpoint_path,f'discriminator_{pgan.n_depth}.png'), show_shapes=True)


Epoch 1/50


2021-12-14 15:11:37.732794: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2021-12-14 15:11:37.868658: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300


270/270 [==============================] - 16s 38ms/step - d_loss: -0.8435 - g_loss: 1.2336
Epoch 2/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.4925 - g_loss: -0.4942
Epoch 3/50
270/270 [==============================] - 8s 29ms/step - d_loss: -0.4655 - g_loss: -0.2741
Epoch 4/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.3270 - g_loss: 0.2009
Epoch 5/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.1676 - g_loss: 0.3189
Epoch 6/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.0903 - g_loss: 0.3525
Epoch 7/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.0924 - g_loss: 0.3153
Epoch 8/50
270/270 [==============================] - 8s 29ms/step - d_loss: -0.0656 - g_loss: 0.2658
Epoch 9/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.1116 - g_loss: 0.2767
Epoch 10/50
270/270 [==============================] - 8s 28ms/step - d_loss: -0.0972 - g_

In [ ]:

# Train faded-in / stabilized generators and discriminators
for n_depth in range(1, 6):



    print(f"[INFO] Fading phase for {n_depth}")
    # Set current level(depth)
    pgan.n_depth = n_depth

    new_dim=2**(n_depth)*4
    new_dim=(new_dim,new_dim)

    ##dataset redefinition
    BS=BS_list[n_depth]
    ts = len(train_imgs) // BS
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)

    train_dataset = (train_dataset.shuffle(1024).map(lambda x: resize(x,new_dim)).cache().repeat().batch(BS).prefetch(AUTOTUNE)
    )
    #enlarge network

    pgan.fade_in_generator()
    pgan.fade_in_discriminator()

    # Draw fade in generator and discriminator
    tf.keras.utils.plot_model(pgan.generator, to_file=opj(checkpoint_path,f'generator_{pgan.n_depth}.png'), show_shapes=True)
    tf.keras.utils.plot_model(pgan.discriminator, to_file=opj(checkpoint_path,f'discriminator_{pgan.n_depth}.png'), show_shapes=True)

    pgan.compile(
      d_optimizer=discriminator_optimizer,
      g_optimizer=generator_optimizer,
    )
    # Train fade in generator and discriminator
    pgan.fit(train_dataset, steps_per_epoch=ts, epochs=EPOCHS_PER_RES, callbacks=callbacks)

    pgan.generator.save_weights(opj(checkpoint_path, f"generator_ndepth_{n_depth}_weights_cifar.h5"))
    pgan.discriminator.save_weights(opj(checkpoint_path, f"discriminator_ndepth_{n_depth}_weights_cifar.h5"))

    try:
        pgan.save_weights(opj(checkpoint_path, f"checkpoint_path_ndepth_{n_depth}_weights_cifar.ckpt"),save_format="tf")
    except:
        print("[WARNING] Could not save weights!")


    print(f"[INFO] Stabilizing phase for {n_depth}")
    pgan.stabilize_generator()
    pgan.stabilize_discriminator()

    # Draw fade in generator and discriminator
    tf.keras.utils.plot_model(pgan.generator, to_file=opj(checkpoint_path,f'generator_{pgan.n_depth}_stabilized.png'), show_shapes=True)
    tf.keras.utils.plot_model(pgan.discriminator, to_file=opj(checkpoint_path,f'discriminator_{pgan.n_depth}_stabilized.png'), show_shapes=True)

    pgan.compile(d_optimizer=discriminator_optimizer,g_optimizer=generator_optimizer,)
    # Train stabilized generator and discriminator
    pgan.fit(train_dataset, steps_per_epoch = ts, epochs = EPOCHS_PER_RES, callbacks=callbacks)
    pgan.generator.save_weights(opj(checkpoint_path, f"generator_stabilized_ndepth_{n_depth}_weights_cifar.h5"))
    pgan.discriminator.save_weights(opj(checkpoint_path, f"discriminator_stabilized_ndepth_{n_depth}_weights_cifar.h5"))

    try:
        pgan.save_weights(opj(checkpoint_path, f"checkpoint_path_stabilized_ndepth_{n_depth}_weights_cifar.ckpt"), save_format="tf")
    except:
        print("[WARNING] Could not save weights!")


[INFO] Fading phase for 1
Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 pixel_normalization (PixelNorm  (None, 256)         0           ['input_2[0][0]']                
 alization)                                                                                       
                                                                                                  
 dense_1 (Dense)                (None, 8192)         2097152     ['pixel_normalization[0][0]']    
                                                                                                  
 weight_scaling_4 (WeightScalin  (None, 8192)        0          